In [ ]:
# sudo chmod 777 /dev/ttyACM0
# sudo chmod 777 /dev/ttyACM1

In [67]:
%matplotlib notebook

# run this in a Jupyter (IPython) Notebook!
# modified from http://www.lebsanft.org/?p=48
# http://pyserial.readthedocs.org/en/latest/pyserial_api.html
import serial
import numpy as np
from matplotlib import pyplot as plt
from time import time

# If you're not using Linux, you'll need to change this
# check the Arduino IDE to see what serial port it's attached to
ser = serial.Serial('/dev/ttyACM0', 115200)

'''
# set plot to animated
plt.ion()
'''

start_time = time()
timepoints = []
ydata = []
yydata = []
yrange = [-0.1,5.1]
view_time = 4 # seconds of data to view at once
duration = 900 # total seconds to collect data

'''
fig1 = plt.figure()
# http://matplotlib.org/users/text_props.html
fig1.suptitle('live updated data', fontsize='18', fontweight='bold')
plt.xlabel('time, seconds', fontsize='14', fontstyle='italic')
plt.ylabel('potential, volts', fontsize='14', fontstyle='italic')
plt.axes().grid(True)
line1, = plt.plot(ydata,marker='o',markersize=4,linestyle='none',markerfacecolor='red')
plt.ylim(yrange)
plt.xlim([0,view_time])
'''

# flush any junk left in the serial buffer
ser.flushInput()
# ser.reset_input_buffer() # for pyserial 3.0+
run = True

# collect the data and plot a moving frame
while run:
    ser.reset_input_buffer()
    data = ser.readline().decode("utf-8").split(' ')
    
    # sometimes the incoming data is garbage, so just 'try' to do this
    try:
        # store the entire dataset for later
        ydata.append(float(data[0]))
        yydata.append(float(data[1]))
        timepoints.append(time()-start_time)
        current_time = timepoints[-1]
        
        # update the plotted data
        line1.set_xdata(timepoints)
        line1.set_ydata(ydata)
        
        '''
        # slide the viewing frame along
        if current_time > view_time:
            plt.xlim([current_time-view_time,current_time])
        '''
            
        # when time's up, kill the collect+plot loop
        if timepoints[-1] > duration: run=False
    
    # if the try statement throws an error, just do nothing
    except: pass
    
'''    
    # update the plot
    fig1.canvas.draw()

# plot all of the data you collected
fig2 = plt.figure()
# http://matplotlib.org/users/text_props.html
fig2.suptitle('complete data trace', fontsize='18', fontweight='bold')
plt.xlabel('time, seconds', fontsize='14', fontstyle='italic')
plt.ylabel('potential, volts', fontsize='14', fontstyle='italic')
plt.axes().grid(True)

plt.plot(timepoints, ydata,marker='o',markersize=4,linestyle='none',markerfacecolor='red')
plt.ylim(yrange)
fig2.show()
'''

ser.close()

In [44]:
ser = serial.Serial('/dev/ttyACM0', 115200)

In [45]:
# flush any junk left in the serial buffer
ser.flushInput()

In [56]:
ser.reset_input_buffer()
data = ser.readline().decode("utf-8").split(' ')
data

['506', '59', '\r\n']

In [66]:
yydata

[140.0,
 232.0,
 19.0,
 158.0,
 38.0,
 175.0,
 197.0,
 114.0,
 68.0,
 188.0,
 109.0,
 120.0,
 80.0,
 102.0,
 47.0,
 102.0,
 143.0,
 142.0,
 79.0,
 160.0,
 52.0,
 3.0,
 124.0,
 114.0,
 32.0,
 70.0,
 18.0,
 189.0,
 123.0,
 116.0,
 190.0,
 247.0,
 56.0,
 17.0,
 157.0,
 230.0,
 3.0,
 139.0,
 79.0,
 204.0,
 66.0,
 22.0,
 167.0,
 208.0,
 141.0,
 155.0,
 125.0,
 158.0,
 16.0,
 54.0,
 157.0,
 56.0,
 53.0,
 118.0,
 49.0,
 163.0,
 35.0,
 84.0,
 116.0,
 30.0,
 193.0,
 22.0,
 211.0,
 24.0,
 89.0,
 251.0,
 223.0,
 42.0,
 123.0,
 228.0,
 43.0,
 13.0,
 211.0,
 160.0,
 178.0,
 15.0,
 154.0,
 233.0,
 126.0,
 200.0,
 14.0,
 30.0,
 19.0,
 234.0,
 106.0,
 32.0,
 185.0]

In [68]:
ydata_par = ydata #ydata[1:][::2] # slicing function
ydata_impar = yydata #ydata[::2] # slicing function

In [ ]:
Q = zip(ydata_par, ydata_impar)

In [69]:
import xlsxwriter

workbook = xlsxwriter.Workbook('hello.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'time')
worksheet.write('B1', 'pwm')
worksheet.write('C1', 'temp')

k  = 1
Ts = 0.025

for pwm, temp in zip(ydata_par, ydata_impar):
    worksheet.write(k, 0, Ts * k)
    worksheet.write(k, 1, pwm)
    worksheet.write(k, 2, temp)
    k += 1
    
workbook.close()